## List of topics

- Class
- Decorator
- Generator
- Context Manager
- Web Scraping
- Data Structure
- Lambda
- Map, Filter, Reduce
- Unpacking: * and **
- args and kwargs


3 interesting features in Python: decorator, generator and context-manager

#### Fundamental pattern 1: Protocol view of Python
- Protocol view of Python: Python Data Model(dunder/magic methods) is a mean by which we can implement protocols. Those protocols have abstract meanings, depending on the object itself. Top level function/syntax -> corresponding __method__
- Built-in inheritance protocol and how it works, where you go on a Python object to learn/look for things 
- Caveats about how OOP in Python works

#### Fundamental pattern 2: Decorator
- Other languages are often upfront. If you make a mistakes in the beginning, very hard to fix
- Python: usually you can write the simplest, stupidest solution. And then later can come back and optimize. A number of features are orientated around how to write the simplest things today and then when the problem gets a little harder, we have an add-in to make the code a bit longer => decorators: example: tracking time with time can be written in a decorator function and add onto other functions that needed to extent that functionality
- take one function and wrap it with additional behavior, taking one aspect, core functionality and wrap a bunch of other functionality in a simple fashion => take the original function, wrap it with a little behavior before and after, like timing, authentication, logging
- you can wrap many functions within functions (higher order decorators)

#### Fundamental pattern 3: Generator
- Eager execution: waits for the entire thing to be computed then give all the results all at once with return => wasteful and time lag can cause trouble in large computation and we might only be interested in the first few results anyway
- Yield/generator doesn't return eagerly but work and return values as it computes (one is storage and eager computation and return => the other is no storage and simply returning value as you use them) => instead of eagerly computing values, you give value to the user as they ask for them (no wait time, no storage) => computation runtime still the same though
- Generators are built upon the idea of code routine: we can interweave code from different code base, a little bit runs in one, a little bit runs in the other. Subroutine: any piece of executable code that runs from beginning to end, one single entry point, one single exit point and that's it. If you look at how code normally interact in user code and library code, if the subroutine is in library code, it has to run to completely until the other runs. For generator, you enter the generator, as you ask for value, it gives it to you => user code run, ask for values in library and generators give it, user code does whatever it wants with value, then come back and ask for another value => interleaving of two pieces of the code
- Generators: a mechanism by which you can interleave codes with other code and also enforce sequences
- Mental model: 
*def api():
*    first_function()
*    yield
*    second_function()
*    yield
*    third_function()
*    yield

#### Fundamental pattern 4: Context Manager
- Resource allocation. There's a corresponding setup and teardown. Some initial action and final action. 
- connect in sql is also a context manager
- The exit is always after the enter => there's sequencing => generator
- Context Manager: some piece of code that setup and tear down actions. Teardown actions always occurs if setup occurs. Allows us to enforce sequencing (exit has to follow setup) and interleaving. We also need sth to adapt generator to the data model we see at the beginning (__method__) => take the generator and wrap it in a method in some fashion => decorator (it's easy to construct dynamic function and have it wrap around other functions) 

#### Expert-level code in Python: 
A clarity of when and where a certain feature might be used. Doesn't waste time and the coder knows that there's a pattern in Python that fit with my purpose. The language provides the core pieces and you need to understand them to assemble the code.
=> need to have a core conceptual understanding of what these features mean
- Python is a language orientated around protocols : some behavior, syntax, top level code, function . And there's a way to tell Python how to implement that on arbitrary objects via __method__. Google python data model => will find all the methods and uses. Code runs from top to bottom. Everything is executable code => you can hook into them, define functions within functions, define class within functions based off of some runtime information that you have. 
- How these impact specific features: metaclass = some hook in the class construction process. Classes are constructed at runtime, you can hook code in there => because you can hook into the creation of classes, you can ask questions like do you have these methods implemented. Simply, you have library and user code, when you're on library side, how do you enforce constraints on user code => some hook into how the classes in the user code are created => where you add metaclass, find the place to add that hook, assert and make sure that the method is implemented
- Generator: take a single computation that would otherwise run eagerly from injection of its parameters to its final computation, and interleave with other codes by adding yield points where the code can yield intermediate values, and also yield control back to the call => can think of it as breaking up one long computation into small parts where each parts is known before the next is computed/run small subunit of that long computation, return partial values or the first values in the sequences of values => offer greater control on how much of that computation to run, whether to run it to completion or just partial way + greater control of memory use as the result is yielded instead of stored in memory 
- Context manager: some structure that allows you to tie 2 actions together (setup action and teardown action) => if setup occurs, make sure teardown also do, even if error occurs in between
- These features are orthogonal but you can combine them in different ways. 

Metaclasses:
- Case of a feature where, if you understand what the feature really is about, it's clear why and when to use it

In [ ]:
#Generator class => already implemented in Python yield

class Gen: 
    def __init__(self, n):
        self.n = n
        self.last = 0 
        
    def __next__(self):
        return self.next()
    
    def next(self):
        if self.last == self.n:
            raise StopIteration()
        rv = self.last ** 2
        self.last += 1
        return rv


In [ ]:
#context manager

from sqlite3 import connect

with connect('test.db') as conn: 
    cur = conn.cursor()
    cur.execute('create table points(x: int, y: int)')
    cur.execute('insert into points (x , y) values(1,1)')
    cur.execute('insert into points (x , y) values(1,2)')
    cur.execute('insert into points (x , y) values(2,1)')
    for row in cur.execute('select x, y from points'):
        print(row)
    for row in cur.execute('select sum(x * y) from points'):
        print(row)
    cur.execute('drop table points')

In [ ]:
#write our own context manager

class temptable:
    
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        print('__enter__')
        self.cur.execute('create table points(x: int, y: int)') #create table
    def __exit__(self, *args):
        print('__exit__')
        self.cur.execute('drop table points') #drop table

with temptable(cur):
    with connect('test.db') as conn: 
        cur = conn.cursor()
        cur.execute('insert into points (x , y) values(1,1)')
        cur.execute('insert into points (x , y) values(1,2)')
        cur.execute('insert into points (x , y) values(2,1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)


In [ ]:
from contextlib import contextmanager #turns decorator into context manager

@contextmanager
def temptable(cur):
    print('__enter__')
    cur.execute('create table points(x: int, y: int)') #create table
    try: 
        yield
    finally: 
        cur.execute('drop table points')
        print('__exit__')

with connect('test.db') as conn: 
    cur = conn.cursor()
    with temptable(cur): 
        cur = conn.cursor()
        cur.execute('insert into points (x , y) values(1,1)')
        cur.execute('insert into points (x , y) values(1,2)')
        cur.execute('insert into points (x , y) values(2,1)')
        for row in cur.execute('select x, y from points'):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)


REAL PYTHON - INTERMEDIATE PYTHON: https://www.youtube.com/watch?v=iba-I4CrmyA&list=PLP8GkvaIxJP0VAXF3USi9U4JnpxUvQXHx&index=8

EXPERT PYTHON: https://www.youtube.com/watch?v=7lmCu8wz8ro

OOP TUTORIAL: https://www.youtube.com/watch?v=RSl87lqOXDE&list=PL-osiE80TeTsqhIuOqKhwlXsIBIdSeYtc&index=4&t=41s

### CLASS AND ITS CONCEPTS

In [19]:
#when to use __str__ and __repr__
#difference: __str__ - representation for end user, giving an easy to read rep of the class
#__repr__ representation for coder, easy to use internally and debug; unambiguous, explicit as possible 
#if __str__ is not written explicitly, when called, it returns __repr__

class Car: 
    
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
        
    def __repr__(self): 
        return '{self.__class__.__name__}({self.color}, {self.mileage})'.format(self = self)
        
    def __str__(self):
        return 'a {self.color} car'.format(self = self)
    
my_car= Car("red", 233000)
my_car
#to see representation of my_car
print(my_car)
str(my_car)

a red car


'a red car'

In [130]:
#Regular methods automatically takes the instance as the first argument => self
# To change this, we use classmethod decorator so the method takes the class as the argument
# Can use classmethod as alternative constructor => use these methods to create multiple ways of creating our objects

#When working with regular methods => it automatically pass the instance as the first argument

#CLASS METHOD automatically pass in the class argument (cls) 
#=> use when you want to provide a simple interface for user when class has complicated constructors that takes a lot of arguments
#=> helps to make the API a little bit easier to work with 
#=> make pizza class: constructors has to pass in lots of ingredients for different pizzas 
#=> create classmethod for different types of pizzas so just need to pass the name of the pizzas in the instance

#STATIC METHOD don't pass anything automatically => behave like regular function, except we include in the class since
#they have some logical connection with the class
#good to create self-contained method that doesn't have much to do with class or the object/instance 
#=> pizza example: static method to calculate the area of the circle of the pizza shape 
#=> easier to maintain the code; people read it in the future knows that this method will not modify the state/behavior of the class

#inheritance allows us to inherit all the attributes and methods of parents class
#we can create subclasses and get all the functionalities of our parents class and then we can
#override or write new functionalities without affecting the parents class in anyway

#property decorator: https://www.youtube.com/watch?v=jCzT9XFZ5bw
#- allow us to define a method where we can access attributes
#- define a method in a class, but it can also act as an attribute

#setter decorator: 
#- ex: @fullname.setter

#deleter decorator: 
#- ex: @fullname.deleter

class Employee:

    num_of_emps = 0
    raise_amt = 1.04

    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.email = first + '.' + last + '@email.com'
        self.pay = pay

        Employee.num_of_emps +=1

    def fullname(self):
        return '{} {}'.format(self.first, self.last)

    def apply_raise(self):
        self.pay = int(self.pay + self.raise_amt)

    @classmethod
    def set_raise_amt(cls, amount):
        cls.raise_amt = amount

    #a function to determine whether it's a workday => has a connection to employee class but doesn't depend on any specfic instance/class variable
    @staticmethod
    def is_workday(day):
        if day.weekday() == 5 or day.weekday() == 6:
            return False
        return True

Employee.set_raise_amt(1.05)

emp_1 = Employee("Corey", "Schafer", 50000)
emp_2 = Employee("Test", "Employee", 60000)

class Developer(Employee):
    
    raise_amt = 1.10
    def __init__(self, first, last, pay, prog_lang):
        super().__init__(first, last, pay)
        self.prog_lang = prog_lang
        
    def __repr__(self):
        return (f'Developer:{self.first} {self.last}, Pay: {self.pay}, Programming Language: {self.prog_lang}')

import datetime

my_date = datetime.date(2016, 7, 10)
print(Employee.is_workday(my_date))

dev_1 = Developer("Corey", "Schafer", 50000, 'Python')
print(dev_1)

False
Developer:Corey Schafer, Pay: 50000, Programming Language: Python


In [34]:
from math import pi

class Pizza: 
    
    def __init__(self, radius, ingredients):
        self.radius = radius
        self.ingredients = ingredients
        
    def __repr__(self):
        return '{self.__class__.__name__}({self.radius}, {self.ingredients})'.format(self=self)
        
    def area(self):
        return self._circle_area(self.radius)
    
    @classmethod
    def margharita(cls, radius):
        return cls(radius,['cheese','tomato', 'basil'])
    
    @staticmethod
    def _circle_area(r):
        return pi*r**2
    
pizza = Pizza(2.5, ['cheese', 'ham'])
print(pizza)
pizza.area()

pizza2 = Pizza.margharita(2.4)
print(pizza2)

Pizza(2.5, ['cheese', 'ham'])
Pizza(2.4, ['cheese', 'tomato', 'basil'])


In [3]:
#if you have two list, use zip to get elements from both 

names = ["Clark Ken", "Peter Parker"]
heroes = ["Superman", "Spiderman"]

for name, hero in zip(names, heroes):
    print(f'{name} is actually {hero}')

Clark Ken is actually Superman
Peter Parker is actually Spiderman


In [4]:
#class implementation
#use setattr and getattr to easily dynamically add attributes to objects in class

In [5]:
#password for security
#import getpass from getpass

In [6]:
#iteritems and xrange: 

#xrange gives us one item at a time instead of loading everything into memory in range()
#iteritems gives us one item at a time instead of loading everything into memory in items()

### Generator

Yield one result at a time => not hold all values in memory => save memory and speed of execution 


In [13]:
def square(nums):
    for i in nums: 
        yield (i*i)

my_num = square([1,2,3,4,5])
next(my_num)

1

In [14]:
for num in my_num: 
    print(num)

4
9
16
25


### Walrus Operator

Assignment expression: “allows you to store and test a value in the same line.”

In [ ]:
#before
nums = [87, 71, 58]
max_range = max(nums) - min(nums)
if max_range > 30: 
    print("Cool")

#with walrus:
nums = [87, 71, 58]
if (max_range := max(nums) - min(nums)) > 30: 
    print("Cool")
    
#before
[determinant(m) for m in matrices if determinant(m) > 0]

#with walrus: if we have a lot of matrices, calculating the determinant twice per matrix could be costly, walrus can help
[d for m in matrices if (d := determinant(m)) > 0]


### String Formating

Techniques: https://therenegadecoder.com/code/how-to-format-a-string-in-python/

Best way is to use f-string

Ex: 

print(f"My name is {name}, and I am {age} years old")

### Switch/Case

Helps to map condition to function in an easy, quick way

func_dict = {
    'condition_a' = handle_a,
    'condition_b' = handle_b,
}

func_dict.get(condition, handle_default)()

In [ ]:
def dispatch_if(operator, x, y):
    if operator == "add":
        return x+y
    elif operator == 'sub':
        return x-y
    elif operator == 'mul':
        return x*y
    elif operator == 'div':
        return x/y
    return None

def dispatch_if_2(operator, x, y):
    return{
        'add': lambda: x + y,
        'sub': lambda: x - y,
        'mul': lambda: x * y,
        'div': lambda: x + y,
    }.get(operator, lambda: None)()

### Function Argument Unpacking (* and ** operators)

- * is for tuple and list
- ** is for dictionary unpacking

In [38]:
gen_exp = (x*x for x in range(3))
#fully exhaust the generator
print(*gen_exp)

0 1 4


In [45]:
dict_gen = {x:y for x in range(5) for y in range(4)}
print(dict_gen)

{0: 3, 1: 3, 2: 3, 3: 3, 4: 3}


In [63]:
#merge dictionary

x = {"a": 1, 'b': 2}
y = {'c': 3, 'd': 4}
z = {**x, **y}
z

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

### Optional Arguments with *args & **kwargs

- args: positional/optional arguments
- kwargs: keyword arguments


1. Can be used for subclassing: take everything that is there previously in the arguments of the parents class and forward it to the arguments in the child class
2. Can be used in decorator function: args and kwargs can be wrapper for these functions and forward arguments to it

In [48]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)
        
foo("hello", 1,2,3,4, key1 = "fun", key2= "done")

hello
(1, 2, 3, 4)
{'key1': 'fun', 'key2': 'done'}


In [50]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
        
class alwaysBlueCar(Car):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.color = 'blue'
                    
#problem is now alwaysBlueCar class has this behavior that the class itself do not control and maintenance might be difficult 
#should put a doc string in alwaysBlueCar to let people know

### Context Manager and the 'with' statement

with statement is good for simplifying some of the common resource management pattern (system resources: files, locks, network connections). 'with' statement can abstract some of the way the function common require like dealing with the acquisition and releasing of these resources back to the system. Usage: 
- open and close database automatically 
- apply and release locks

Context manager: a simple protocol/interface/contract that your object follows so that they can be used with the 'with' statement

In [51]:
#files => with statement makes sure the file will be closed after we're done with it

with open('helloworld.txt', 'w') as f:
    f.write("Hello World")
    
#bad way
f = open('helloworld.txt', 'w')
try: 
    f.write("Hello World")
finally:
    f.close()

In [52]:
#2 methods needed in an object to be supported with the 'with': __enter__ and __exit__

class ManagedFile:
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()
            
with ManagedFile('helloworld.txt') as f: 
    f.write("Hello World")

In [53]:
from contextlib import contextmanager

@contextmanager
def OpenFile(file, mode):
    try: 
        f = open(file, mode)
        yield f
    finally:
        f.close()
        
with OpenFile('sample.txt', 'w') as f: 
    f.write("Today's awesome")

print(f.closed) 

### Virtual Environment

Packaging system to manage the dependencies of programs so other modules that the program uses that are not part of the standard library and we don't write ourselves. Pip install things in the global environment which might create conflicts in different projects that require different versions of the modules => Virtualenv allows use to keep all the dependencies of the projects, including different versions of the dependencies
=> create an isolated python env with all the packages and dependencies

https://www.youtube.com/watch?v=UqkT2Ml9beg

- Create new project
- Change to the directory of that project with: cd
- Create virtual environment: python3 -m venv ./venv #create VE inside the python project
- Check the VE with: ls
- Check the tree structure of VE: tree ./venv
- Activate the VE for the project: source ./venv/bin/activate
- Check if everything's run in the virtual environment: which pip3
- Now when you install things, you will go in the virtual env instead of global
- See what is installed in a package: pip3 list
- Install new module for the virtual env: pip3 install schedule
- Check if the new module is installed inside virtualenv: pip3 list
- Go back to global environment: deactivate

### Pipenv - Manage Package and Virtual Environments

Combine pip and venv

### Importlib - New Version of Modules

In case when the modules import changes, we can bring the changes in with importlib

In [ ]:
import importlib
importlib.reload(awesome.py)

### Unittest and Doctest - Testing code

Allows for developers to create a class of tests that are run and analyzed flexibly: 
- Create a test class as a subclass of the unittest.TestCase type,
- Add tests as methods of this class, make sure that name of each test functions begins with 'test'
- Add unittest.main() to main loop to run

In [ ]:
import unittest

class FactorialTest(unittest.TestCase):
    def testSingleValue(self):
        self.assertEqual(factorial(5), 120)
        
    def testMultipleValues(self):
        self.assertRaise(TypeError, factorial, [1,2,3,4])
        
    def testBoolean(self):
        self.assertTrue(factorial(5) == 120)
        
def main():
    unittest.main()

if __name__ == "__main__":
    main()

### Profiling

cProfiler - easiest is to call the module with the python file in the terminal:
- python -m cProfiler myscript.py

In [ ]:
import cProfiler 

# some functions

cProfiler.run(#function)

In [104]:
nonprimes = [j for i in range(2,19) for j in range(i**2, 500, i)]
nonprimes = sorted(nonprimes)
nonprimes

[4,
 6,
 8,
 9,
 10,
 12,
 12,
 14,
 15,
 16,
 16,
 18,
 18,
 20,
 20,
 21,
 22,
 24,
 24,
 24,
 25,
 26,
 27,
 28,
 28,
 30,
 30,
 30,
 32,
 32,
 33,
 34,
 35,
 36,
 36,
 36,
 36,
 38,
 39,
 40,
 40,
 40,
 42,
 42,
 42,
 44,
 44,
 45,
 45,
 46,
 48,
 48,
 48,
 48,
 49,
 50,
 50,
 51,
 52,
 52,
 54,
 54,
 54,
 55,
 56,
 56,
 56,
 57,
 58,
 60,
 60,
 60,
 60,
 60,
 62,
 63,
 63,
 64,
 64,
 64,
 65,
 66,
 66,
 66,
 68,
 68,
 69,
 70,
 70,
 70,
 72,
 72,
 72,
 72,
 72,
 74,
 75,
 75,
 76,
 76,
 77,
 78,
 78,
 78,
 80,
 80,
 80,
 80,
 81,
 81,
 82,
 84,
 84,
 84,
 84,
 84,
 85,
 86,
 87,
 88,
 88,
 88,
 90,
 90,
 90,
 90,
 90,
 91,
 92,
 92,
 93,
 94,
 95,
 96,
 96,
 96,
 96,
 96,
 98,
 98,
 99,
 99,
 100,
 100,
 100,
 100,
 102,
 102,
 102,
 104,
 104,
 104,
 105,
 105,
 105,
 106,
 108,
 108,
 108,
 108,
 108,
 110,
 110,
 110,
 111,
 112,
 112,
 112,
 112,
 114,
 114,
 114,
 115,
 116,
 116,
 117,
 117,
 118,
 119,
 120,
 120,
 120,
 120,
 120,
 120,
 120,
 121,
 122,
 123,
 124,
 124,


### Share packages on PyP

In [ ]:
from setuptools import setup

setup(name = "pyTest",
      version = "0.0.1",
      description = "Awesome Python Package",
      author = "Dang Ngoc Huy",
      author_email = "huydang90@gmail.com",
      packages = ['feeling']
     )

### Underscores (_ & __) in Python variable names

_ : by convention, a hint to another programmer that the variable is meant to be treated as a private variable by the programmer

__: name mangling, change the name of that variable in a way that's gonna make it harder to create collision when someone else extends the class => ex: if you create a subclass and extend the inheritance, the name mangling prevents collision of naming in that subclass in the case when you use similar/same name 


### Type Hint

Syntax extension added in Python 3.5, helps to specify the expected types of the argument and expected return type

In [54]:
def add(a: int, b: int) -> int:
    return a + b

### Customs Exception

Hints for why something goes wrong

In [55]:
class NameTooShort(ValueError):
    pass

def validate(name):
    if len(name) < 10:
        raise NameTooShort(name)

### Data Structure

Sets: 
- take up more space but 
- provide much faster membership checkup (since it's backed by dictionary - hash table)
- doesn't allow duplicates
- no order
- is dynamic, so can add new element to set with method .add() as long as it is hashable

Set comprehension, similar to list comprehension

In [59]:
squares = {x*x for x in range(10)}
squares

{0, 1, 4, 9, 16, 25, 36, 49, 64, 81}

In [60]:
#use set constructor to create empty set. Use {} will just create dictionary
new_set = set()
new_set.add(5)
new_set

{5}

In [61]:
#frozenset => cannot modify, immutable, compeletely static => useful when you need a constant set
vowels = frozenset({a, e, i, o, u})

froze = frozenset()
froze.add(5)

AttributeError: 'frozenset' object has no attribute 'add'

In [62]:
#multisets (bags) => allows to have duplicates and knows how many times it is included in the set
import collections 
inventory = collections.Counter()
loot = {'bread': 2, 'sword': 1}
inventory.update(loot)
more_loot = {'bread': 3, 'sword': 1, 'shoes': 1}
inventory.update(more_loot)
inventory

Counter({'bread': 5, 'sword': 2, 'shoes': 1})

### Functional Programming

A programming technique that avoid side effects of the program by performing mathematical computations by mainly through the evaluation of functions. Heavily use immutable data structures => reduce potential for bugs


In [64]:
import collections
Scientist = collections.namedtuple('Scientist', ['name', 'field', 'born', 'nobel'])
ada = Scientist(name = "Ada Lovelace", field = 'math', born = 1895, nobel = False)
ada.name

'Ada Lovelace'

In [65]:
scientist = [
    Scientist(name = "Ada Lovelace", field = 'math', born = 1895, nobel = False), 
    Scientist(name = "Emme Noeathe", field = 'math', born = 1895, nobel = True)
]

from pprint import pprint #pretty print
pprint(scientist)

[Scientist(name='Ada Lovelace', field='math', born=1895, nobel=False),
 Scientist(name='Emme Noeathe', field='math', born=1895, nobel=True)]


### OS modules

The OS module allows us to interact wiht the underlying operating system in several different ways.

- Navigate the file system
- Get file information
- Look up and change the environment variables
- Move files around
- Many more

To begin, import the os module. This is a built in module, no third party modules need to be installed.

https://www.youtube.com/watch?v=tJxcKyFMTGo

In [121]:
import os

print(os.getcwd())

os.chdir('/Users/dangngochuy/Desktop/')

os.makedirs('OS-Module/sub-modules')
os.removedirs('OS-Module/sub-modules')

print(os.getcwd())
print(os.listdir())



# Get current working directory
os.getcwd()


# Change directory, this requires a path to change to
os.chdir(path)


# List directory, you can pass a path, but by default it is in the current directory
os.listdir()


# Multiple options for creating directories
mkdir()  # Use for making one directory
makedirs(). # Use if you want to create multiple directories at once


# Remove directories
rmdir(file). # Recommended use case
removedirs(file)  # Removes intermediate directories if specified


# Rename a file or folder
os.rename(‘test.txt’, ‘demo.txt’). # This renames text.txt to demo.txt


# Look at info about files
os.stat(test.txt)
# Useful stat results: st_size (bytes), st_mtime (time stamp)


# To see entire directory tree and files within
# os.walk is a generator that yields a tuple of 3 values as it walks the directory tree

for dirpath, dirnames, filenames in os.walk(routepath): 
    print(‘Current Path:’, dirpath)
    print(‘Directories:’, dirnames)
    print(‘Files:’, filenames)
    print()

# This is useful for locating a file that you can’t remember where it was
# If you had a web app, and you wanted to keep track of the file info within a certain directory structure, then you could to thru the os.walk method and go thru all files and folders and collect file information.


# Access home directory location by grabbing home environment variable
os.environ.get(‘HOME’). # Returns a path
# To properly join two files together use os.path.join()
file_path = os.path.join(os.environ.get(‘HOME’), ‘test.txt’)
# the benefit of os.path.join, is it takes the guess work out of inserting a slash


# os.path has other useful methods

os.path.basename
# This will grab filename of any path we are working on

os.path.dirname(‘/tmp/test.txt’)
# returns the directory /tmp

os.path.split(‘/tmp/test.txt’)
# returns both the directory and the file as a tuple

os.path.exists(‘/tmp/test.txt’)
# returns a boolean

os.path.isdir(‘/tmp/test.txt’)
# returns False

os.path.isfile(‘/tmp/test.txt’)
# returns True

os.path.splitext(‘/tmp/test.txt’)
# Splits file route of the path and the extension
# returns (‘/tmp/test’, ‘.txt’)
# This is alot easier than parsing out the extension. Splitting off and taking the first value is much better.
# Very useful for file manipulation



/Users/dangngochuy/Desktop
/Users/dangngochuy/Desktop
['vmls.pdf', 'sams-teach-yourself-sql-in-24-hours-4th-edition.pdf', 'Thoả thuận hợp tác TDC 2020 VN.docx', 'Screen Shot 2020-02-03 at 05.29.36.png', 'Home', 'photo.html', '~$ng-Thanh-Tung-Resume-VIETNAMESE.docx', 'Screen Shot 2019-12-12 at 23.59.45.png', 'Data Structure in Python', '~$a hồng.docx', 'Fluent Python.pdf', '~$thon Datacamp.docx', '~$EADING III & IV.docx', 'Screen Shot 2020-01-24 at 01.01.21.png', 'CommandLine.pdf', '.DS_Store', 'Berlin Days', 'Learning_Python.pdf', '~$̀i liệu 1 (1).docx', 'certificate.pdf', '~$i phat bieu_Huy dich (Autosaved).docx', '~$tro to TensorFlow for AI.docx', '~$3889 Public Management Policy Brief .docx', '400627f26be965c5473c5b5b8c1fc29e.png', 'Hertie', '.localized', '80971951_2527050100901485_3094656015602286592_o.png', '~$thon development.docx', 'chapter3.pdf', '21016172_10154686844466746_8623838640201089402_o.jpg', '[PROGRAMMING][Clean Code by Robert C Martin].pdf', '~$prove Deep N

### Request Module

https://www.youtube.com/watch?v=tb8gHvYlCFs

### Json Module

https://www.youtube.com/watch?v=9N6a-VLBa2I

### Web Scraping 

https://www.youtube.com/watch?v=ng2o98k983k&t=11s

### LEGB - Local Enclosing Global Built-Ins

Scoping of variables in the LEGB order

In [133]:
def outer():
    x = 'outer x'
    
    def inner():
        nonlocal x # x now becomes local x of the enclosing function outer()
        x = 'inner x' #override x value
        print(x)
    
    inner()
    print(x) #'inner x' will be printed twice when outer() is called

outer()

inner x
inner x


### EAFP - Easier to ask for forgiveness, not permission 

Try and do something. If it works, great. If not, throw an error. 
=> it's faster (you access the object immediately, without having to check again and again)
=> avoid raiseerror situation when we can

LBYL - Look before you leap (hasattr method)

In [138]:
class Duck: 
    """Create a duck"""
    
    def quack(self):
        print("Quack Quack")
        
    def fly(self):
        print("Flap Flap")
        
        
class Person: 
    """Create a person"""
    
    def quack(self):
        print("I'm Quack")
        
    def fly(self):
        print("I'm Flap")

        
def quack_and_fly(thing):
    # LBYL (Non-Pythonic)
    #if hasattr(thing, 'quack'):
        #if callable(thing.quack):
            #thing.quack

    if hasattr(thing, 'fly'):
        if callable(thing.fly):
            thing.fly
            
    #EAFP (Pythonic)
    try: 
        thing.quack()
        thing.fly()
        thing.bark()
    except AttributeError as e: 
        print(e)
        
    print()

d = Duck()
quack_and_fly(d)

p = Person()
quack_and_fly(p)

Quack Quack
Flap Flap
'Duck' object has no attribute 'bark'

I'm Quack
I'm Flap
'Person' object has no attribute 'bark'



### Threading and Multi-threading

Run Code Concurrently Using the Threading Module:
- CPU-bound (things that are crunching a lot of numbers and using CPU) and IO-bound (things that are just waiting for input and output operations to be completed and not using CPU all that much: reading, writing from the file systems, network operation - downloading stuff online) tasks
- Threading is beneficial when it comes to IO-bound tasks - a lot of waiting for input and output operations 
- Not a lot of benefit for CPU-bound due to the overhead of creating and destroying threads => use multiprocessing and run in parallel instead
- When using concurrency with threads: not gonna run code at the same time, just the illusion of doing so as when it comes to a point of just waiting around, it's just gonna move forward with the script and run other codes while the IO operation finishes => so the task can be done a bit sooner and the wait time can be much shorter 

Video: https://www.youtube.com/watch?v=IEEhzQoKtQU

In [145]:
import threading
import time

start = time.perf_counter()

def do_sth():
    print("Sleep for 1 second\n")
    time.sleep(1)
    print("Done sleeping\n")
    
t1 = threading.Thread(target = do_sth)
t2 = threading.Thread(target = do_sth)

t1.start()
t2.start()

t1.join()
t2.join() #so the two threads runs almost at the same time

end = time.perf_counter()

print(f'Finished in {round(end-start,2)} second(s)')

Sleep for 1 second

Sleep for 1 second

Done sleeping
Done sleeping


Finished in 1.01 second(s)


In [146]:
#looping threads for multiple tasks

start = time.perf_counter()


threads = []

for _ in range(10):
    t = threading.Thread(target = do_sth)
    t.start()
    threads.append(t) # so they're all in a list and run concurrently
    
for thread in threads: 
    thread.join()
    
end = time.perf_counter()

print(f'Finished in {round(end-start,2)} second(s)')

Sleep for 1 second

Sleep for 1 second
Sleep for 1 second


Sleep for 1 second

Sleep for 1 second

Sleep for 1 second

Sleep for 1 second
Sleep for 1 second

Sleep for 1 second

Sleep for 1 second


Done sleeping
Done sleeping
Done sleeping



Done sleeping

Done sleeping

Done sleeping
Done sleeping


Done sleeping
Done sleeping
Done sleeping



Finished in 1.01 second(s)


In [148]:
#use arguments

start = time.perf_counter()

def do_sth(seconds):
    print(f"Sleep for {seconds} second\n")
    time.sleep(seconds)
    print("Done sleeping\n")

for _ in range(10):
    t = threading.Thread(target = do_sth, args = [1.5])
    t.start()
    threads.append(t)
    
for thread in threads: 
    thread.join()
    
end = time.perf_counter()

print(f'Finished in {round(end-start,2)} second(s)')

Sleep for 1.5 second
Sleep for 1.5 second


Sleep for 1.5 second

Sleep for 1.5 second
Sleep for 1.5 second


Sleep for 1.5 second

Sleep for 1.5 second

Sleep for 1.5 second

Sleep for 1.5 second

Sleep for 1.5 second

Done sleeping
Done sleeping
Done sleeping



Done sleeping
Done sleeping


Done sleeping
Done sleeping
Done sleeping
Done sleeping

Done sleeping




Finished in 1.52 second(s)


In [150]:
#new way to do it => best to use with a context manager

import concurrent.futures

start = time.perf_counter()

def do_sth(seconds):
    print(f"Sleep for {seconds} second\n")
    time.sleep(seconds)
    return "Done sleeping\n"

with concurrent.futures.ThreadPoolExecutor() as executor:
    f1 = executor.submit(do_sth, 1)
    print(f1.result())
    
#run thread multiple times with list comprehension
with concurrent.futures.ThreadPoolExecutor() as executor:
    #secs = [5,4,3,2,1]
    results = [executor.submit(do_sth, 1) for _ in range(10)]
    #results = [executor.submit(do_sth, sec) for sec in secs]
    
    for f in concurrent.futures.as_completed(results):
        print(f.result())
        
end = time.perf_counter()

print(f'Finished in {round(end-start,2)} second(s)')

Sleep for 1 second

Done sleeping

Sleep for 1 second

Sleep for 1 second

Sleep for 1 second

Sleep for 1 second
Sleep for 1 second

Sleep for 1 second
Sleep for 1 second



Sleep for 1 second
Sleep for 1 second


Sleep for 1 second

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Finished in 2.02 second(s)


### Multiprocessing

Speed comes from running different tasks in parallel

In [151]:
import multiprocessing

start = time.perf_counter()

p1 = multiprocessing.Process(target = do_sth)
p2 = multiprocessing.Process(target = do_sth)

p1.start()
p2.start()

p1.join()
p2.join()
                             
end = time.perf_counter()

print(f'Finished in {round(end-start,2)} second(s)')

#looping same as threading

Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/dangngochuy/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/dangngochuy/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/dangngochuy/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/dangngochuy/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
TypeError: do_sth() missing 1 required positional argument: 'seconds'
TypeError: do_sth() missing 1 required positional argument: 'seconds'


Finished in 0.13 second(s)


In [154]:
#Multiprocessing Pool 

start = time.perf_counter()

results = []
with concurrent.futures.ProcessPoolExecutor() as executor: 
    secs = [5,4,3,2,1]
    results = executor.map(do_sth, secs)
    for result in results: 
        print(result)

end = time.perf_counter()

print(f'Finished in {round(end-start,2)} second(s)')

Sleep for 4 second
Sleep for 2 second
Sleep for 3 second
Sleep for 5 second




Sleep for 1 second

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Done sleeping

Finished in 5.08 second(s)


### Decorators 

How to use: 
- allow us to easily add functionality to our existing functions by adding that functionality inside of our wrapper function without modifying our original function 
- use: create a logging decorator to log activities for debugging; create timer to time functions 

In [159]:

def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print(f"wrapper executed this before {original_function.__name__}")
        return original_function(*args, **kwargs)
    return wrapper_function #ready to be executed but not executed immediately at runtime

def display():
    print("Original")
    
decorated_display = decorator_function(display)
decorated_display() # now it's executed

wrapper executed this before display
Original


In [161]:
#anytime we use @decorator, then the function that follows will have the added-on functionality of the decorator function

@decorator_function
def display():
    print("Original")
    
@decorator_function
def display_info(name, age): 
    print(f"{name} {age}")

display()
display_info("John", 20)

wrapper executed this before display
Original
wrapper executed this before display_info
John 20


In [167]:
from functools import wraps

def my_logger(original_function):
    import logging
    logging.basicConfig(filename = f'{original_function.__name__}.log',level = logging.INFO)
    
    @wraps(original_function)
    def wrapper(*args, **kwargs):
        logging.info(
        f'Ran with args: {args} and kwargs: {kwargs}')
        return original_function(*args, **kwargs)
    return wrapper 
            
def my_timer(original_function):
    import time
    
    @wraps(original_function)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time()
        print(f'{original_function.__name__} runs in {t2-t1} seconds')
        return result
    return wrapper

@my_logger #stacked decorators
@my_timer
def display_info(name, age): 
    print(f"{name} {age}")
    
display_info("John", 20)

John 20
display_info runs in 0.01336812973022461 seconds


### NamedTuples

In [170]:
from collections import namedtuple

Color = namedtuple('Color', ['red', 'green', 'blue'])
color = Color(55, 200, 100)
print(color.red)
print(color[0])
white = Color(255, 255, 255)

55
55


### Request 

Request library - easy to make request for http information from websites: post info, download images, send authentication info, basic log-in and forms, follow, redirect.
- Great to get info, but not for parsing info (beautifulsoup is good for this)

In [26]:
import requests

r = requests.get('https://xkcd.com/353/')
print(r) # response object
print(dir(r)) #see the methods available => there's a content method

<Response [200]>
['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']


In [28]:
help(r.content)

Help on bytes object:

class bytes(object)
 |  bytes(iterable_of_ints) -> bytes
 |  bytes(string, encoding[, errors]) -> bytes
 |  bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
 |  bytes(int) -> bytes object of size given by the parameter initialized with null bytes
 |  bytes() -> empty bytes object
 |  
 |  Construct an immutable array of bytes from:
 |    - an iterable yielding integers in range(256)
 |    - a text string encoded using the specified encoding
 |    - any object implementing the buffer API.
 |    - an integer
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(...)
 |  
 |  __g

In [29]:
print(r.text)

<!DOCTYPE html>
<html>
<head>
<link rel="stylesheet" type="text/css" href="/s/b0dcca.css" title="Default"/>
<title>xkcd: Python</title>
<meta http-equiv="X-UA-Compatible" content="IE=edge"/>
<link rel="shortcut icon" href="/s/919f27.ico" type="image/x-icon"/>
<link rel="icon" href="/s/919f27.ico" type="image/x-icon"/>
<link rel="alternate" type="application/atom+xml" title="Atom 1.0" href="/atom.xml"/>
<link rel="alternate" type="application/rss+xml" title="RSS 2.0" href="/rss.xml"/>
<script type="text/javascript" src="/s/b66ed7.js" async></script>
<script type="text/javascript" src="/s/1b9456.js" async></script>

<meta property="og:site_name" content="xkcd">

<meta property="og:title" content="Python">
<meta property="og:url" content="https://xkcd.com/353/">
<meta property="og:image" content="https://imgs.xkcd.com/comics/">
<meta name="twitter:card" content="summary_large_image">

</head>
<body>
<div id="topContainer">
<div id="topLeft">
<ul>
<li><a href="/archive">Archive</a></li>
<l

In [34]:
new_r = requests.get('https://imgs.xkcd.com/comics/python.png')

with open('comic.png', 'wb') as f: #create context manager to write the byte content of the requested file
    f.write(new_r.content) #content is downloaded into the directory

In [38]:
print(new_r.status_code) #200 is success, 300 is redirect, 400 is client error, 500: server error

200


In [ ]:
print(new_r.ok) #returns True if status code is < 400

In [40]:
#Test different HTTP method: https://httpbin.org/

payload = {'page': 2, 'count': 25} # dict of params to put into requested link 

r = requests.get('https://httpbin.org/get', params = payload) #equivalent: https://httpbin.org/get/?page=2&count=25
print(r.text)

{
  "args": {
    "count": "25", 
    "page": "2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.21.0", 
    "X-Amzn-Trace-Id": "Root=1-5e424baa-fa4eba9983ac5a8243673804"
  }, 
  "origin": "149.233.47.215", 
  "url": "https://httpbin.org/get?page=2&count=25"
}



In [41]:
print(r.url) #correct url parameters creation

https://httpbin.org/get?page=2&count=25


In [44]:
#Post to route

payload = {'user': 'corey', 'pass': 'testing'} # dict of params to put into requested link 

r = requests.post('https://httpbin.org/post', data = payload) #equivalent: https://httpbin.org/get/?page=2&count=25
r_dict = r.json() #create a dict from that json response and can be used as any dict in Python
print(r_dict['form']) #same as importing json modules then using json.load on text

{'pass': 'testing', 'user': 'corey'}


In [45]:
#Test authentication - specify creditials in the url, pass different creditials to the route and see if it works 

r = requests.get('https://httpbin.org/basic-auth/corey/testing', auth = ('corey', 'testing')) 
print(r.text)
print(r)

{
  "authenticated": true, 
  "user": "corey"
}

<Response [200]>


In [47]:
r = requests.get('https://httpbin.org/basic-auth/corey/testing', auth = ('coreyms', 'testing')) #wrong user
print(r.text)
print(r)


<Response [401]>


In [48]:
#timeout for cases when request takes too long

r = requests.get('https://httpbin.org/delay/6', timeout=3) #wrong user
print(r)

ReadTimeout: HTTPSConnectionPool(host='httpbin.org', port=443): Read timed out. (read timeout=3)

### JSON - Javascript Object Notation

- Common data format for storing info, especially when you fetch info from API, also used for configuration files and different data that can be saved on local machines. 
- Inpisred by Javascript but now independent of any language => any language has a parser for json


In [56]:
import json

people_string = '''
{
    "people": [
    {
        "name": "John Smith",
        "phone": "01332323", 
        "email": ["johnsmith@gmail.com", "john@gmail.com"],
        "has_license": false
    }, 
    {
        "name": "Jane Smith",
        "phone": "01332323", 
        "email": null,
        "has_license": true
    }

    ]
}
'''

data = json.loads(people_string) #convert json object into equivalent python object
print(data)
print(type(data))

{'people': [{'name': 'John Smith', 'phone': '01332323', 'email': ['johnsmith@gmail.com', 'john@gmail.com'], 'has_license': False}, {'name': 'Jane Smith', 'phone': '01332323', 'email': None, 'has_license': True}]}
<class 'dict'>


In [57]:
for person in data['people']: #each person is a dict which we can access info from
    print(person)
    print(person['name'])

{'name': 'John Smith', 'phone': '01332323', 'email': ['johnsmith@gmail.com', 'john@gmail.com'], 'has_license': False}
John Smith
{'name': 'Jane Smith', 'phone': '01332323', 'email': None, 'has_license': True}
Jane Smith


In [58]:
# dump python object into json object

for person in data['people']:
    del person['phone']
    
new_string = json.dumps(data, indent = 2, sort_keys = True) #indent helps to format and makes json easier to read

print(new_string)

{
  "people": [
    {
      "name": "John Smith",
      "email": [
        "johnsmith@gmail.com",
        "john@gmail.com"
      ],
      "has_license": false
    },
    {
      "name": "Jane Smith",
      "email": null,
      "has_license": true
    }
  ]
}


In [ ]:
#open files 

import json

with open('states.json') as f:
  data = json.load(f)

for state in data['states']:
  del state['area_codes']

with open('new_states.json', 'w') as f:
  json.dump(data, f, indent=2)


In [ ]:
#grabbing json from a public api 
#yahoo api that converts usd into other currencies

import json
from urllib.request import urlopen

with urlopen("https://finance.yahoo.com/webservice/v1/symbols/allcurrencies/quote?format=json") as response:
    source = response.read()

data = json.loads(source)

print(json.dumps(data, indent=2))

usd_rates = dict()

for item in data['list']['resources']:
    name = item['resource']['fields']['name']
    price = item['resource']['fields']['price']
    usd_rates[name] = price

print(50 * float(usd_rates['USD/INR'])) #convert 50usd to euros


### SQLite

SQLite allows us to quickly get up and running with databases, without spinning up larger databases like MySQL or Postgres. 
- Can use for small and medium size application where your database is gonna live on disc 
- Prototyping and testing application before needing to move to a larger database
- SQLite can use memory that lives in RAM



In [61]:

class Employee:
    """A sample Employee class"""

    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.pay = pay

    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)

    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)

    def __repr__(self):
        return "Employee('{}', '{}', {})".format(self.first, self.last, self.pay)


In [69]:
#create an database to delete and add employees and grab info of employees from that database

import sqlite3
from employee import Employee

conn = sqlite3.connect(':memory:')

c = conn.cursor()

c.execute("""CREATE TABLE employees (
            first text,
            last text,
            pay integer
            )""")


def insert_emp(emp):
    with conn:
        c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp.first, 'last': emp.last, 'pay': emp.pay})


def get_emps_by_name(lastname):
    c.execute("SELECT * FROM employees WHERE last=:last", {'last': lastname})
    return c.fetchall()


def update_pay(emp, pay):
    with conn:
        c.execute("""UPDATE employees SET pay = :pay
                    WHERE first = :first AND last = :last""",
                  {'first': emp.first, 'last': emp.last, 'pay': pay})


def remove_emp(emp):
    with conn:
        c.execute("DELETE from employees WHERE first = :first AND last = :last",
                  {'first': emp.first, 'last': emp.last})

emp_1 = Employee('John', 'Doe', 80000)
emp_2 = Employee('Jane', 'Doe', 90000)

insert_emp(emp_1)
insert_emp(emp_2)

emps = get_emps_by_name('Doe')
print(emps)

update_pay(emp_2, 95000)
remove_emp(emp_1)

emps = get_emps_by_name('Doe')
print(emps)

conn.close()

('Corey', 'Shafer', 50000)


### Webscraping with BeautifulSoup



In [72]:
from bs4 import BeautifulSoup
import requests

with open('simple.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')
    
match = soup.find('div', class_ = 'footer')
print(match)

<div class="footer">
<p>Footer Information</p>
</div>


In [77]:
for article in soup.find_all('div', class_ = 'article'):

    headline = article.h2.a.text
    print(headline)

    summary = article.p.text
    print(summary)
    
    print()

Article 1 Headline
This is a summary of article 1

Article 2 Headline
This is a summary of article 2



In [81]:
#scrape website
source = requests.get('https://coreyms.com/').text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())

In [88]:
#grab first article: 

article = soup.find('article')
#print(article.prettify())
headline = article.h2.a.text
print(headline)
 
print()

summary = article.find('div', class_ = 'entry-content').p.text
print(summary)

print()

vid_src = article.find('iframe', class_ = 'youtube-player')['src'] #get yt link
vid_id = vid_src.split('/')[4]
vid_id = vid_id.split('?')[0]
print(vid_id)

yt_link = f'https://youtube.com/watch?v={vid_id}'

Python Tutorial: Zip Files – Creating and Extracting Zip Archives

In this video, we will be learning how to create and extract zip archives. We will start by using the zipfile module, and then we will see how to do this using the shutil module. We will learn how to do this with single files and directories, as well as learning how to use gzip as well. Let’s get started…

z0gguhEmWiY


In [93]:
#grab all 
import csv

csv_file = open('csm_scrape.csv', 'w')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headline', 'summary', 'video'])

for article in soup.find_all('article'):
    headline = article.h2.a.text
    print(headline)

    summary = article.find('div', class_ = 'entry-content').p.text
    print(summary)
    try: 
        vid_src = article.find('iframe', class_ = 'youtube-player')['src'] #get yt link
        vid_id = vid_src.split('/')[4]
        vid_id = vid_id.split('?')[0]
        yt_link = f'https://youtube.com/watch?v={vid_id}'
    except:
        yt_link = None
        
    print(yt_link)
    
    print()
    
    csv_writer.writerow([headline, summary, yt_link])
    
csv_file.close()

Python Tutorial: Zip Files – Creating and Extracting Zip Archives
In this video, we will be learning how to create and extract zip archives. We will start by using the zipfile module, and then we will see how to do this using the shutil module. We will learn how to do this with single files and directories, as well as learning how to use gzip as well. Let’s get started…
https://youtube.com/watch?v=z0gguhEmWiY

Python Data Science Tutorial: Analyzing the 2019 Stack Overflow Developer Survey
In this Python Programming video, we will be learning how to download and analyze real-world data from the 2019 Stack Overflow Developer Survey. This is terrific practice for anyone getting into the data science field. We will learn different ways to analyze this data and also some best practices. Let’s get started…
https://youtube.com/watch?v=_P7X8tMplsw

Python Multiprocessing Tutorial: Run Code in Parallel Using the Multiprocessing Module
In this Python Programming video, we will be learning how t

In [96]:
def lower_common_hypernym():
  first = input("Enter word 1: ")
  second = input("Enter word 2: ")
  m_first = wn.synset(first)[0]
  print(m_first.hypernyms())
  m_second = wn.synset(second)[0]
  print(m_second.hypernyms())
lower_common_hypernym()

Enter word 1: love
Enter word 2: dog


NameError: name 'wn' is not defined

### Practice Challenges

In [72]:
# Please write a program to randomly generate a list with 5 numbers, which are divisible by 5 and 7 , between 1 and 1000 inclusive.

import random

print(random.sample([x for x in range(1,1001) if (x%5 == 0) and (x%7 ==0)], 5))

[315, 420, 385, 735, 665]


In [76]:
# Please write a program to shuffle and print the list [3,6,7,8].
li = [3,6,7,8]
random.shuffle(li)
print(li)

[6, 7, 3, 8]


In [87]:
#Write a program to generate and print another tuple whose values are even numbers in the given tuple (1,2,3,4,5,6,7,8,9,10). 

gen = [x for x in (1,2,3,4,5,6,7,8,9,10) if x % 2 == 0]
new_tup = tuple(gen)
new_tup

(2, 4, 6, 8, 10)

In [94]:
#Write a program to solve a classic ancient Chinese puzzle: 
#We count 35 heads and 94 legs among the chickens and rabbits in a farm. 
#How many rabbits and how many chickens do we have?

def count_animals(heads, legs): 
    rabbit = int(legs/2-heads)
    chicken = int(heads - rabbit)
    return f'Rabbits: {rabbit}, Chickens: {chicken}'
    
count_animals(35, 94)

UsageError: Line magic function `%%timeit` not found.


In [ ]:
def solve(numheads,numlegs):
    ns='No solutions!'
    for i in range(numheads+1):
        j=numheads-i
        if 2*i+4*j==numlegs:
            return i,j
    return ns,ns

In [106]:
#Please write a program which prints all permutations of [1,2,3]

import itertools
print(list(itertools.permutations([1,2,3])))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]


In [118]:
n = 8
{i: i*i for i in range(1, n+1)}

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49, 8: 64}

In [2]:
#Please write a program which accepts a string from console and print it in reverse order.

def reversed_order():
    new_str = input()
    new_str = new_str[::-1]
    return new_str

reversed_order()

halloween


'neewollah'

In [6]:
#Please write a program which accepts a string from console and print the characters that have even indexes.

def even_index():
    new_str = input()
    new_str = new_str[::2]
    return new_str
even_index()

noway


'nwy'

In [8]:
#Please write a program which count and print the numbers of each character in a string input by console.

def count_char():
    new_str = input()
    new_dict = {}
    count = 0
    for char in new_str:
        if char in new_dict:
            new_dict[char] += 1
        else: 
            new_dict[char] = 1
    print(new_dict)
    
count_char()

mywayisyourway
{'m': 1, 'y': 4, 'w': 2, 'a': 2, 'i': 1, 's': 1, 'o': 1, 'u': 1, 'r': 1}


In [9]:
#Define a class Person and its two child classes: Male and Female. All classes have a method "getGender" which can print "Male" for Male class and "Female" for Female class.

class Person: 
    
    def getGender(self):
        return unknown
    
class Male(Person):
    
    def getGender(self):
        return "Male"
        
class Female(Person):

    def getGender(self):
        return "Female"

    

In [16]:
#With a given list [12,24,35,24,88,120,155,88,120,155], write a program to print this list after removing all duplicate values with original order reserved.
 
def rm_dup(ls):
    new_ls = []
    new_set = set()
    for item in ls: 
        if item not in new_set:
            new_set.add(item)
            new_ls.append(item)
    return new_ls
    
ls = [12,24,35,24,88,120,155,88,120,155]
rm_dup(ls)

[12, 24, 35, 88, 120, 155]

In [17]:
# With two given lists [1,3,6,78,35,55] and [12,24,35,24,88,120,155], write a program to make a list whose elements are intersection of the above given lists.

def intersect(l1, l2):
    """Take 2 lists"""
    s1 = set(l1)
    s2 = set(l2)
    inter = s1.intersection(s2)
    return list(inter)

l1 = [1,3,6,78,35,55]
l2 = [12,24,35,24,88,120,155]

intersect(l1, l2)

[35]

In [18]:
#By using list comprehension, please write a program to print the list after removing the value 24 in [12,24,35,24,88,120,155].

print([x for x in [12,24,35,24,88,120,155] if x != 24])

[12, 35, 88, 120, 155]


In [19]:
#By using list comprehension, please write a program to print the list after removing the 0th,4th,5th numbers in [12,24,35,70,88,120,155].
li = [12,24,35,70,88,120,155]
print([x for (i,x) in enumerate(li) if i not in [0,4,5]])

[24, 35, 70, 155]


In [20]:
#By using list comprehension, please write a program generate a 3*5*8 3D array whose each element is 0.

print([[[0 for col in range(8)] for col in range(5)] for row in range(3)])

[[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]]


In [22]:
#Please write a program to generate all sentences where subject is in ["I", "You"] and verb is in ["Play", "Love"] and the object is in ["Hockey","Football"].

subjects = ["I", "You"]
vers = ["Play", "Love"]
objects = ["Hockey","Football"]
for i in range(len(subjects)):
    for j in range(len(vers)):
        for k in range(len(objects)): 
            print(f'{subjects[i]} {vers[j]} {objects[k]}')



I Play Hockey
I Play Football
I Love Hockey
I Love Football
You Play Hockey
You Play Football
You Love Hockey
You Love Football


In [ ]:
# Please write a program to compress and decompress the string "hello world!hello world!hello world!hello world!".



In [100]:
import inspect
import random

inspect.getdoc(random.randrange)

'Choose a random item from range(start, stop[, step]).\n\nThis fixes the problem with randint() which includes the\nendpoint; in Python this is usually not what you want.'